In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
liocinema_subs = pd.read_csv(r'/content/subscription.csv')
liocinema_content = pd.read_csv(r'/content/content.csv')
liocinema_content_cons = pd.read_csv(r'/content/content_consumption.csv')

jotstar_subs = pd.read_csv(r'/content/jotstar_subs.csv')
jotstar_content = pd.read_csv(r'/content/jotstar_content.csv')
jotstar_content_cons = pd.read_csv(r'/content/jotstar_content_cons.csv')

/tmp/ipython-input-3-277541346.py:1: DtypeWarning: Columns (5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  liocinema_subs = pd.read_csv(r'/content/subscription.csv')


In [4]:
liocinema_subs.head()

,user_id,age_group,city_tier,subscription_date,subscription_plan,last_active_date,plan_change_date,new_subscription_plan
0,UIDLC0009f86256c,18-24,Tier 3,12-11-2024,Free,NaN,NaN,NaN
1,UIDLC00a7c713f21,18-24,Tier 3,12-11-2024,Free,NaN,NaN,NaN
2,UIDLC02567e3e1c9,18-24,Tier 3,12-11-2024,Free,NaN,NaN,NaN
3,UIDLC031be258d26,18-24,Tier 3,12-11-2024,Free,NaN,NaN,NaN
4,UIDLC039ce0641e5,18-24,Tier 3,12-11-2024,Free,NaN,NaN,NaN


In [5]:
liocinema_subs = liocinema_subs.drop(columns = ['last_active_date', 'plan_change_date',
       'new_subscription_plan'])

jotstar_subs = jotstar_subs.drop(columns = ['last_active_date', 'plan_change_date',
       'new_subscription_plan'])

In [6]:
jotstar_subs['subscription_plan'] = jotstar_subs['subscription_plan'].replace('Premium','Premium_1')

In [7]:
df_result = pd.concat([liocinema_subs,jotstar_subs] ,axis = 0).reset_index().drop(columns = ['index'])

In [8]:
df_result.head()

,user_id,age_group,city_tier,subscription_date,subscription_plan
0,UIDLC0009f86256c,18-24,Tier 3,12-11-2024,Free
1,UIDLC00a7c713f21,18-24,Tier 3,12-11-2024,Free
2,UIDLC02567e3e1c9,18-24,Tier 3,12-11-2024,Free
3,UIDLC031be258d26,18-24,Tier 3,12-11-2024,Free
4,UIDLC039ce0641e5,18-24,Tier 3,12-11-2024,Free


Now Let's introduce a weighted price for the new ott service.

In [9]:
old_price_basic = 69
old_price_prem = 129
old_price_prem_1 = 359
old_price_vip = 159


##New average of basic plan is weighted average.
new_basic = int(((53362 * 69) + (129 * 25092))/ (53362 + 25092))
new_basic

##same with premium plan
new_prem = int(((19157 * 159) + (129 * 25092))/ (25092 + 19157))
new_prem

##ultimate rounded off plans are these.
new_basic = 90
new_prem = 140
new_vip = 360

Base Model based on the weighted average.

In [10]:
## How many records are in basic
df_result_basic = df_result[df_result['subscription_plan'] == 'Basic']

## Taking the numbers of
n_users = int(len(df_result_basic) * abs( (90-60) /90))

##Resanpling users
resampled_users = df_result_basic.sample(n = n_users, random_state = 42)

df_result.loc[resampled_users.index,'subscription_plan'] = 'Free'

In [11]:
df_result['subscription_plan'] = df_result['subscription_plan'].replace('Premium_1','Premium')

In [12]:
df_result.head()

,user_id,age_group,city_tier,subscription_date,subscription_plan
0,UIDLC0009f86256c,18-24,Tier 3,12-11-2024,Free
1,UIDLC00a7c713f21,18-24,Tier 3,12-11-2024,Free
2,UIDLC02567e3e1c9,18-24,Tier 3,12-11-2024,Free
3,UIDLC031be258d26,18-24,Tier 3,12-11-2024,Free
4,UIDLC039ce0641e5,18-24,Tier 3,12-11-2024,Free


In [13]:
df_result['subscription_date'] = pd.to_datetime(df_result['subscription_date'],format="%d-%m-%Y")
df_result['month'] =11 -  pd.to_datetime(df_result['subscription_date']).dt.month

In [14]:
##Total revenue from the sevice
df_result['total_revenue'] = 0


##Total revenue is number of month subscribed from multiplied by the plan payment.
df_result.loc[df_result['subscription_plan'] == 'Premium', 'total_revenue' ] = df_result['month'] * new_prem
df_result.loc[df_result['subscription_plan'] == 'Basic', 'total_revenue' ] = df_result['month'] * new_basic
df_result.loc[df_result['subscription_plan'] == 'VIP', 'total_revenue' ] = df_result['month'] * new_vip

In [19]:
a = df_result['total_revenue'].sum()

In [20]:
f'Total Revenue generated by This subscription model :- {round(a/1000000,2)} Million'

'Total Revenue generated by This subscription model :- 65.06 Million'

In [21]:
user_on_each_plan = df_result['subscription_plan'].value_counts()
user_on_each_plan

,count
subscription_plan,
Free,134875
Premium,38459
Basic,35575
VIP,19157


## We will skip it for now.

Overwhelming Majority lies on free tier. Let's try to deriev methods to attract them.

In [35]:
## Here i calculated the elasticity rate and price drop with my own assumptions.
elastic_rate = 1.1

perc_change_in_price = -1 *  0.05 # of all plans

no_of_users_gained =  elastic_rate*perc_change_in_price
-1 * no_of_users_gained * df_result.shape[0]

12543.630000000001

### Let's create a model and try to get revenue from the city tier 3.

In [36]:
df = df_result.groupby(['subscription_plan','city_tier'])['city_tier'].count()
df = df.to_frame()
df =  df.rename(columns={'city_tier': 'city_tier_1'})
df = df.reset_index()
df_free = df[df['subscription_plan'] == 'Free']
df_free.head()

,subscription_plan,city_tier,city_tier_1
3,Free,Tier 1,27633
4,Free,Tier 2,43753
5,Free,Tier 3,63489


In [37]:
ex = px.pie(df_free, names = 'city_tier',values = 'city_tier_1', title = 'Age wise distribution')
ex.show()

so apparantly 18 - 25 age group mostly wants to watch Hindi- English(predom. this) content lagnuage

In [38]:
df_analysis = df_result.drop(columns = ['user_id','subscription_date',
                                       'month'])

In [39]:
df_analysis['age_group'] = df_analysis['age_group'].map({'18-24':0,'25-34':1,'35-44':2,'45+':3})
df_analysis['city_tier'] = df_analysis['city_tier'].map({'Tier 3': 0,'Tier 2':1,'Tier 1':2})
df_analysis['subscription_plan'] = df_analysis['subscription_plan'].map({'Free': 0,'Basic':1,'Premium':2,'VIP':3})

In [40]:
df_1 = df_result.groupby('city_tier')['city_tier'].count()
df_1 = df_1.to_frame()
df_1 = df_1.rename(columns = {'city_tier':'count'})
df_1 = df_1.reset_index()
df_1.head()

,city_tier,count
0,Tier 1,66462
1,Tier 2,77272
2,Tier 3,84332


In [41]:
## Here we filtered out our tier 3 people into a different dataframe
df_res = df_result[df_result['city_tier'] == 'Tier 3']

df_filtered = df_result[df_result['city_tier'] != 'Tier 3']

In [42]:
a = df_res['total_revenue'].sum()

f'Total Revenue generated by This subscription model :- {round(a/1000000,2)} Million from tier 3 cities.'

'Total Revenue generated by This subscription model :- 10.6 Million from tier 3 cities.'

In [43]:
# Set constant PED
ped = 1.5

# Define price drops for each migration
price_drop_dict = {
    'Premium_to_VIP': -0.10,   # 20% drop
    'Basic_to_Premium': -0.10, # 15% drop
    'Free_to_Basic': -0.10     # 25% drop
}

In [67]:
# Premium → VIP
df_result_prem_change = df_res[df_res['subscription_plan'] == 'Premium']  ## select peoplw eith that particular plan
perc_change_qty = ped * price_drop_dict['Premium_to_VIP']       ## Assign new the number of people who shift to new plan a variable
new_userss = int(len(df_result_prem_change) * abs(perc_change_qty))     ## randomly samples people who shift to upper plan
resampled = df_result_prem_change.sample(n=new_userss, random_state=42)     ##
df_res.loc[resampled.index, 'subscription_plan'] = 'VIP'


# Basic → Premium
df_result_basic_change = df_res[df_res['subscription_plan'] == 'Basic']
perc_change_qty = ped * price_drop_dict['Basic_to_Premium']
new_userss = int(len(df_result_basic_change) * abs(perc_change_qty))
resampled = df_result_basic_change.sample(n=new_userss, random_state=12)
df_res.loc[resampled.index, 'subscription_plan'] = 'Premium'

# Free → Basic
df_result_free_change = df_res[df_res['subscription_plan'] == 'Free']
perc_change_qty = ped * price_drop_dict['Free_to_Basic']
new_userss = int(len(df_result_free_change) * abs(perc_change_qty))
resampled = df_result_free_change.sample(n=new_userss, random_state=22)
df_res.loc[resampled.index, 'subscription_plan'] = 'Basic'


In [45]:
new_basic = 90
new_prem = 140
new_vip = 360


new_model_basic = new_basic * (0.90)
new_model_vip = new_vip * (0.95)
new_model_prem = new_prem * (0.95)

In [46]:
##Total revenue from the sevice
df_res['month'] = 11 - pd.to_datetime(df_res['subscription_date']).dt.month

df_res['total_revenue'] = 0

df_res.loc[df_res['subscription_plan'] == 'Premium', 'total_revenue' ] = df_res['month'] * new_model_prem
df_res.loc[df_res['subscription_plan'] == 'Basic', 'total_revenue' ] = df_res['month'] * new_model_basic
df_res.loc[df_res['subscription_plan'] == 'VIP', 'total_revenue' ] = df_res['month'] * new_model_vip

/tmp/ipython-input-46-1315279259.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipython-input-46-1315279259.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [47]:
df_combined = pd.concat([df_filtered, df_res]).sort_index()
a1 = df_result['total_revenue'].sum()
b1 = df_combined['total_revenue'].sum()
(b1-a1)/ a1 * 100

np.float64(4.499801796237925)

#### We found 4.5 % growth in revenue in part to price specific model in tier 3 cities of overall model. Giving them 10% off will give us 4.5% increase in profits.

In [48]:
df_combined.head()

,user_id,age_group,city_tier,subscription_date,subscription_plan,month,total_revenue
0,UIDLC0009f86256c,18-24,Tier 3,2024-11-12,Free,0,0
1,UIDLC00a7c713f21,18-24,Tier 3,2024-11-12,Free,0,0
2,UIDLC02567e3e1c9,18-24,Tier 3,2024-11-12,Free,0,0
3,UIDLC031be258d26,18-24,Tier 3,2024-11-12,Free,0,0
4,UIDLC039ce0641e5,18-24,Tier 3,2024-11-12,Basic,0,0


## Now, let's see 18 - 24 age group sub. How can we turn them around.

In [49]:
age_prople = df_combined.groupby(['age_group','subscription_plan'])['subscription_plan'].count()

## About 40% of people are from age-range 18 - 24. Yet, 21% of total from are in free tier.

In [175]:
df_age = df_combined[df_combined['age_group'] == '18-24']

df_rest_age = df_combined[df_combined['age_group'] != '18-24']

In [178]:
df_age['total_revenue'].sum()

np.int64(18117901)

In [179]:
perc_change_in_price = -0.5
PED = 1.1
perc_change_in_demand = PED * perc_change_in_price
perc_change_in_demand


-0.55

In [180]:
# Conversion assumptions
perc_change_in_price = -0.5

# Step 1: Free → Basic (high PED)
PED_basic = 1.1
perc_change_in_demand_basic = PED_basic * perc_change_in_price
existing_early_users = df_age[df_age['subscription_plan'] == "Free"]
new_users_basic = int(len(existing_early_users) * abs(perc_change_in_demand_basic))
resampled_basic = df_age.sample(n=new_users_basic, random_state=42)
df_age.loc[resampled_basic.index, 'subscription_plan'] = 'Basic'

# Step 2: Free → Premium (medium PED)
PED_premium = 0.3 # more conservative
perc_change_in_demand_premium = PED_premium * perc_change_in_price
new_users_premium = int(len(existing_early_users) * abs(perc_change_in_demand_premium))
resampled_premium = df_age.sample(n=new_users_premium, random_state=43)
df_age.loc[resampled_premium.index, 'subscription_plan'] = 'Premium'

# Step 3: Free → VIP (low PED)
PED_vip = 0.6
PED_premium = 0.3  # most conservative
perc_change_in_demand_vip = PED_vip * perc_change_in_price
new_users_vip = int(len(existing_early_users) * abs(perc_change_in_demand_vip))
resampled_vip = df_age.sample(n=new_users_vip, random_state=44)
df_age.loc[resampled_vip.index, 'subscription_plan'] = 'VIP'

# Step 4: Pricing & Revenue assignment
new_basic = 90
new_prem = 140
new_vip = 360

new_price_basic = new_basic * 0.5
new_price_premium = new_prem * 0.5
new_price_vip = new_vip * 0.5

df_age.loc[df_age['subscription_plan'] == 'Basic', 'total_revenue'] = df_age['month'] * new_price_basic
df_age.loc[df_age['subscription_plan'] == 'Premium', 'total_revenue'] = df_age['month'] * new_price_premium
df_age.loc[df_age['subscription_plan'] == 'VIP', 'total_revenue'] = df_age['month'] * new_price_vip


In [183]:
df_age['total_revenue'].sum()

np.int64(19285500)

In [186]:
(19285500 - 18117901)/18117901 * 100

6.444449608152733

### 6% increase in profit from 18-24 age range people